In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import json
def open_jsonl_file(jsonl_path):
    data = []
    with open(jsonl_path, 'r',encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [ ]:
train_data = open_jsonl_file('../../Datasets/hikr_org_train_test_valid/train.jsonl')
test_data = open_jsonl_file('../../Datasets/hikr_org_train_test_valid/test.jsonl')
valid_data = open_jsonl_file('../../Datasets/hikr_org_train_test_valid/valid.jsonl')

In [ ]:
import numpy as np

def get_single_slope(lat,lon):
    point = ee.Geometry.Point([lon, lat])
    if lat >= 60:
        dem = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic').select('elevation')
    else:
        dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
    slope = ee.Terrain.slope(dem)
    point_feature_collection = ee.FeatureCollection(point)
    sampled_features = slope.sampleRegions(collection=point_feature_collection)
    sampled_dict = sampled_features.getInfo()
    if sampled_dict['features'] == []:
        return np.nan
    else:
        slope = sampled_dict['features'][0]['properties']['slope']
        return slope

In [ ]:
def get_multi_slope(track):
    slope_list = []
    latitudes = track['latitude']
    longitudes = track['longitude']
    features = [ee.Feature(ee.Geometry.Point([lon, lat])) for lat, lon in zip(latitudes, longitudes)]
    feature_collection = ee.FeatureCollection(features)
    first_lat = latitudes[0]
    first_lon = longitudes[0]
    if first_lat >= 60:
        dem = ee.Image('UMN/PGC/ArcticDEM/V3/2m_mosaic').select('elevation')
    else:
        dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
    slope = ee.Terrain.slope(dem)
    sampled_features = slope.sampleRegions(collection=feature_collection)
    sampled_dict = sampled_features.getInfo()
    for feature in sampled_dict['features']:
        slope = feature['properties']['slope']
        slope_list.append(slope)
    return slope_list

In [ ]:
from tqdm import tqdm
import os

def add_slope_to_track(inputpath, savepath):
    os.makedirs(os.path.dirname(savepath), exist_ok=True)
    data = open_jsonl_file(inputpath)
    with open(savepath, 'w', encoding='utf-8') as f:
        for track in tqdm(data):
            slope_list = get_multi_slope(track)
            if len(track["latitude"]) != len(slope_list):
                slope_list = []
                for lat, lon in zip(track["latitude"], track["longitude"]):
                    slope = get_single_slope(lat, lon)
                    slope_list.append(slope)
            track["Terrain_slope(degrees)"] = slope_list
            f.write(json.dumps(track, ensure_ascii=False) + '\n')

In [ ]:
file_types = ['train', 'test', 'valid']
for file_type in file_types:
    input_file = f"../../Datasets/hikr_org_train_test_valid/{file_type}.jsonl"
    output_file = f"../../Datasets/processed_dataset/{file_type}.jsonl"
    add_slope_to_track(input_file, output_file)

In [ ]:
def calculate_terrain_slope_stats(filepath):
    slopes = []
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            if "Terrain_slope(degrees)" in data:
                slopes.extend(data["Terrain_slope(degrees)"])
    mean = np.mean(slopes)
    std = np.std(slopes)
    return mean, std

In [ ]:
def add_stats_to_config(config_path, mean, std):
    try:
        with open(config_path, 'r') as file:
            config = json.load(file)

        config['Terrain_slope(degrees)'] = {'mean': mean, 'std': std}

        with open(config_path, 'w') as file:
            json.dump(config, file, indent=4)
    except Exception as e:
        print(e)

In [ ]:
import shutil
def create_processed_config(source_path, target_path, mean, std):
    shutil.copy(source_path, target_path)
    
    with open(target_path, 'r') as file:
        config = json.load(file)
    config['Terrain_slope(degrees)'] = {'mean': mean, 'std': std}
    
    with open(target_path, 'w') as file:
        json.dump(config, file, indent=4)

In [ ]:
source_config_path = '../../Datasets/hikr_org_train_test_valid/config.json'
target_config_path = '../../Datasets/processed_dataset/config.json'
mean, std = calculate_terrain_slope_stats('../../Datasets/processed_dataset/train.jsonl')

create_processed_config(source_config_path, target_config_path, mean, std)